In [2]:
# import libraries
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import seaborn as sns
import matplotlib.pyplot as plt

from data_structures import Signal, Portfolio

In [54]:
import pandas as pd
import numpy as np

# Set the seed for reproducibility
np.random.seed(42)

# Specify the number of rows (10 or 20)
num_rows = 10  # or num_rows = 20

# Create a DataFrame with random numbers
data = {'Cash': [2,21,3,0,51,55,1,3,5,4],
        'Sim Portfolio Value': np.random.randint(1, 100, num_rows),
        'SP5E': [1,1,1,1,1,1,1,3,5,4],
        'R3V': np.random.randint(1, 100, num_rows),
        'SP5': np.random.randint(1, 100, num_rows)}

df = pd.DataFrame(data)

# Display the DataFrame
df

,Cash,Sim Portfolio Value,SP5E,R3V,SP5
0,2,52,1,88,64
1,21,93,1,24,60
2,3,15,1,3,21
3,0,72,1,22,33
4,51,61,1,53,76
5,55,21,1,2,58
6,1,83,1,88,22
7,3,87,3,30,89
8,5,75,5,38,49
9,4,75,4,2,91


In [55]:
# find point when cash first runs out
full = (df['Cash'] == 0).idxmax()
benchmarks = ['SP5E', 'R3V', 'SP5']

for i in benchmarks:
    # set starting value to sim portfolio value when cash runs out
    df[f'{i} Portfolio Value'] = df.at[full, 'Sim Portfolio Value']
    # create growth factor column
    growth_factor = df[i] / df.at[full, i]
    # create value column
    df[f'{i} Portfolio Value'] = df[f'{i} Portfolio Value'] * growth_factor

In [56]:
df

,Cash,Sim Portfolio Value,SP5E,R3V,SP5,SP5E Portfolio Value,R3V Portfolio Value,SP5 Portfolio Value
0,2,52,1,88,64,72.0,288.000000,139.636364
1,21,93,1,24,60,72.0,78.545455,130.909091
2,3,15,1,3,21,72.0,9.818182,45.818182
3,0,72,1,22,33,72.0,72.000000,72.000000
4,51,61,1,53,76,72.0,173.454545,165.818182
5,55,21,1,2,58,72.0,6.545455,126.545455
6,1,83,1,88,22,72.0,288.000000,48.000000
7,3,87,3,30,89,216.0,98.181818,194.181818
8,5,75,5,38,49,360.0,124.363636,106.909091
9,4,75,4,2,91,288.0,6.545455,198.545455


### Understanding Price Movements

In [7]:
price_data_start = 14
selected_columns = [0, 2] + list(range(price_data_start, 2511))

df = pd.read_excel(r'C:\Users\MarcoHui\Desktop\SimulatorData 50ATS OLD VERSION 35.xlsx', sheet_name='prices', skiprows=1, usecols=selected_columns)

# reduce signals to improve plot readability
df = df.head(50)

In [ ]:
value_vars = df.columns[3:]
temp = pd.melt(df, id_vars='SourceDateNam', value_vars=value_vars, var_name='Days After Signal Date', value_name='Price')

fig, ax = plt.subplots(figsize=(80, 10))

plot = sns.lineplot(
    data=temp,
    x='Days After Signal Date',
    y='Price',
    hue='SourceDateNam',
    ax=ax
)

### Testing Portfolio Simulator Mechanisms

In [43]:
# set up
temp = df[46:49]
my_portfolio = Portfolio(5)

# buy signals
my_portfolio.buyFromDf(temp, 1)

# check that they have been bought
print('\nAFTER FIRST BUY')
my_portfolio.display()
print('cash: ', str(my_portfolio.cash))


# update price and value
my_portfolio.updateMonthlyPrice()
my_portfolio.updateMonthlyVal()

# check that price and values have been updated
print('\nAFTER ONE PRICES AND VALUES UPDATE')
my_portfolio.display()
print('cash: ', str(my_portfolio.cash))

# update price and value until expiry
for i in range(157):
    my_portfolio.updateMonthlyPrice()
    my_portfolio.updateMonthlyVal()

# check that price and values have been updated
print('\nAFTER MANY PRICES AND VALUES UPDATES')
my_portfolio.display()
print('cash: ', str(my_portfolio.cash))

# dump expired
my_portfolio.dumpExpired()

# print sell list
print('\nWHAT WE DUMPED')
print(my_portfolio.solddf)

# check that expired signals have been sold at last available price
print('\nAFTER EXPIRED DUMPED')
my_portfolio.display()
print('cash: ', str(my_portfolio.cash))

# buy more signals
temp = df[49:50]
my_portfolio.buyFromDf(temp, 4)

# check that cash is gone and oldest stock has been replaced
print('\nAFTER ANOTHER BUY')
my_portfolio.display()
print('cash: ', str(my_portfolio.cash))

# print sell list
print('\nWHAT WE SOLD TO FUND NEW SIGNAL')
print(my_portfolio.solddf)


AFTER FIRST BUY
jna34139817The    pricelast: na    priceprev/now/next: 33.71921 33.83364 33.66199    growth: 0    val: 1
zach72139818thi    pricelast: 10.45    priceprev/now/next: 5.9 5.98 5.65    growth: 0    val: 1
jna34139819GOO    pricelast: na    priceprev/now/next: 8.3515 8.05025 8.12975    growth: 0    val: 1
cash:  2

AFTER ONE PRICES AND VALUES UPDATE
jna34139817The    pricelast: na    priceprev/now/next: 33.83364 33.66199 33.29962    growth: -0.005073353029706518    val: 0.9949266469702935
zach72139818thi    pricelast: 10.45    priceprev/now/next: 5.98 5.65 8.34    growth: -0.05518394648829432    val: 0.9448160535117057
jna34139819GOO    pricelast: na    priceprev/now/next: 8.05025 8.12975 7.87675    growth: 0.009875469705909688    val: 1.0098754697059096
cash:  2

AFTER MANY PRICES AND VALUES UPDATES
jna34139817The    pricelast: na    priceprev/now/next: 39.92712 39.85084 39.85084    growth: -0.0019104808961929666    val: 1.1778466638528993
zach72139818thi    pricelast: 10.